## Lesson Notebook 6: Machine Translation

In this notebook we will look at several components:

   * Simple translation examples with T5 

   * Translation example with M2M100 - many more languages

   * Transformer encoder decoder translation of Shakespeare to Modern English

   * MT metrics examples

   * Subword models and tokenizers

Part 4 reuses the transformer code from the Keras Tutorial https://keras.io/examples/nlp/neural_machine_translation_with_transformer/


<a id = 'returnToTop'></a>

## Notebook Contents
  * 1. [Setup](#setup)
  * 2. [Simple Translation Model](#simpleTranslation)
  * 3. [M2M100 Translation Example](#m2mTranslation)
  * 4. [Shakespeare-to-Modern English translation with a sequence-to-sequence Transformer](#shakespeare)
  * 5. [Machine Translation Metrics](#translationMetrics)  
  * 6. [Subword Models](#subwordModels)
  * [Answers](#answers)







  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2022-fall-main/blob/master/materials/lesson_notebooks/lesson_6_Machine_translation.ipynb)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

## 1. Setup


We'll start with the usual setup. We need to begin with the sentencepiece code in order to tokenize the text for some of the models.

In [1]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 20.5 MB/s 


In [2]:
!pip install -q transformers


     |████████████████████████████████| 4.9 MB 31.3 MB/s 
     |████████████████████████████████| 6.6 MB 59.4 MB/s 
     |████████████████████████████████| 120 kB 50.2 MB/s 


In [3]:
!pip install -q datasets

     |████████████████████████████████| 431 kB 37.8 MB/s 
     |████████████████████████████████| 115 kB 73.6 MB/s 
     |████████████████████████████████| 212 kB 68.8 MB/s 
     |████████████████████████████████| 127 kB 46.0 MB/s 


In [4]:
#Am I running a GPU and what type is it?
!nvidia-smi

Fri Sep 23 20:57:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[Return to Top](#returnToTop)  
<a id = 'simpleTranslation'></a>


## 2. Simple Translation Example

These T5 models are trained to translate in one direction only.  For example, they can translate from English to French but not from French to English. 

Let's test this out.

In [5]:
#import T5 and show 
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [6]:
SENTENCE_TO_TRANSLATE = ( "PG&E stated it scheduled the blackouts in response to forecasts for high winds \
            amid dry conditions.")

BACK_TRANSLATE_TEST = ("PG&E a déclaré qu'elle avait prévu les panne de courant.")

In [7]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-base') #also t5-small and t5-large
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

t5_model.summary()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  84954240  
                                                                 
 decoder (TFT5MainLayer)     multiple                  113275008 
                                                                 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Add the prompt to the sentence we want to translate so the model knows what we want it to do with the input.

In [8]:
t5_input_text = "translate english to french: " + SENTENCE_TO_TRANSLATE

In [9]:
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

**QUESTION 1**: What do the inputs look like?  We've already seen BERT inputs. What's happening with T5? What's the same as what we saw with BERT and what's different? 

In [10]:
t5_inputs

{'input_ids': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[13959, 22269,    12, 20609,    10,     3,  7861,   184,   427,
         4568,    34,  5018,     8,  1001,   670,     7,    16,  1773,
           12,  7555,     7,    21,   306, 13551, 18905,  2192,  1124,
            5,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [11]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'])
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1602: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


["PG&E a déclaré qu'elle avait prévu les panne de courant"]


Not bad. Now let's try the reverse even though we know the model wasn't trained to translate in that direction.  What do you think it will do?

In [12]:
t5_back_text = "translate french to english: " + BACK_TRANSLATE_TEST

In [13]:
t5_binputs = t5_tokenizer([t5_back_text], return_tensors='tf')

The decoder still runs and emits language, specifically French, as requested.  These models will pretty much always produce some output but you need to make sure that you're asking it to do something it can and that it is doing the right thing.

In [14]:
t5_summary_ids = t5_model.generate(t5_binputs['input_ids'])
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

["PG&E a déclaré qu'elle avait prévu les panne de courant"]


[Return to Top](#returnToTop)  
<a id = 'm2mTranslation'></a>


## 3. M2M100 Translation Example

M2M100 is a large model that was pre-trained on many languages simultaneoulsy.  You do need to give it some clues about what you are expecting when it translates.  Typically this takes the form of specifying the input and the output languages.  Let's look at the tokenizer first. How can it handle 100 different languages?

In [15]:
from transformers import M2M100Config, M2M100Tokenizer

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="fr")

tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁Transform',
 'ers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

Now let's try to translate.  We have two original sentences, one in English and one in Chinese that have roughly the same meaning.

In [16]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

en_text = "Do not meddle in the affairs of wizards, for they are subtle and quick to anger."
chinese_text = "不要插手巫師的事務, 因為他們是微妙的, 很快就會發怒."

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="zh")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

Downloading:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

In [17]:
encoded_zh = tokenizer(chinese_text, return_tensors="pt")

Let's start by taking the Chinese sentence and translate it back in to English.

In [18]:
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['Do not interfere with the matters of the witches, because they are delicate and will soon be angry.']

Interesting and subtly different from our English original. Now we'll try translating the English to Chinese and then we'll take that Chinese output and translate it back into English.  This should give us an idea of how well the model works.

In [19]:
encoded_en = tokenizer(en_text, return_tensors="pt")

In [20]:
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("zh"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。']

Now we'll store that Chinese output in a variable so we can translate back to English.

In [21]:
chinese_back_text = '不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。'

In [22]:
encoded_zhb = tokenizer(chinese_back_text, return_tensors="pt")

In [23]:
generated_tokens = model.generate(**encoded_zhb, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Do not interfere with the things of the witches, because they are delicate and pleasant anger.']

Now you can see how far it has drifted as we have translated back and forth. With some care this approach can be used to generate novel content that can augment a training set (as long as the drift isn't too bad). This is what we call back translation.

[Return to Top](#returnToTop)  
<a id = 'shakespeare'></a>

## 4. Shakespeare-to-Modern English translation with a sequence2sequence Transformer

In this example, we'll build a sequence-to-sequence Transformer model, which
we'll train on a Shakespearian English to Modern English machine translation task.

You'll learn how to:

- Preprocess and vectorize text using the Keras `TextVectorization` layer.
- Implement a `TransformerEncoder` layer, a `TransformerDecoder` layer,
and a `PositionalEmbedding` layer.
- Prepare data as sentence pairs for training a sequence-to-sequence machine translation model.
- Use the trained model to generate translations.

How does this differ from using T5 or M2M100?  This model is not pre-trained so it knows nothing about language when we start to train it.


In [24]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

### 4.1 Downloading the data

The data includes aligned sentences from a number of plays by William Shakespeare.  The data was copied from this repo --[https://github.com/cocoxu/Shakespeare](https://github.com/cocoxu/Shakespeare) -- and consolidated into one file for easier handling.

You will to grab a copy from our git repo and import it to your Google drive.  From there you'll be able to easily load it in to a Colab notebook.

In [25]:
#This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#Modify this path to the appropriate location in your Drive
text_file = 'drive/MyDrive/data/train_plays-org-mod.txt'

### 4.2 Parsing the data

Each line contains a Shakespearean sentence and its corresponding modern English translation.
The Shakesperean sentence is the *source sequence* and modern English one is the *target sequence*.
We prepend the token `"[start]"` and we append the token `"[end]"` to the modern English sentence.

In [27]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    org, mod = line.split("\t")
    mod = "[start] " + mod + " [end]"
    text_pairs.append((org, mod))

In [28]:
#look at some examples
for _ in range(5):
    print(random.choice(text_pairs))

('I do not think it good.', '[start] I don’t think it’s a good idea. [end]')
('Sir, in my heart there was a kind of fighting That would not let me sleep.', '[start] Sir, there was a kind of fighting in my heart That wouldn’t let me sleep. [end]')
('rebels it at these years?', '[start] It rebels at this age? [end]')
('Hail to your lordship!', '[start] Greetings to your lordship! [end]')
('True.', '[start] True. [end]')


In [29]:
#Let's create some splits
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

19088 total pairs
13362 training pairs
2863 validation pairs
2863 test pairs


Note we have roughly 13,000 sentence pairs.  How does this compare with the number of sentences in other sentence pair corpora we've seen?

### 4.3 Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for Shakespeare and one for modern English),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace), while
the Shakespearean layer will use a custom standardization, where we also specify the set of punctuation characters to be stripped.

Note: in a production-grade machine translation model, I would not recommend
stripping the punctuation characters in either language. Instead, I would recommend turning
each punctuation character into its own token,
which you could achieve by providing a custom `split` function to the `TextVectorization` layer.

In [30]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 32


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


org_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)

mod_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_org_texts = [pair[0] for pair in train_pairs]
train_mod_texts = [pair[1] for pair in train_pairs]
org_vectorization.adapt(train_org_texts)
mod_vectorization.adapt(train_mod_texts)

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the vectorized source sentence and `encoder_inputs` is the target sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

In [31]:

def format_dataset(org, mod):
    org = org_vectorization(org)
    mod = mod_vectorization(mod)
    return ({"encoder_inputs": org, "decoder_inputs": mod[:, :-1],}, mod[:, 1:])


def make_dataset(pairs):
    org_texts, mod_texts = zip(*pairs)
    org_texts = list(org_texts)
    mod_texts = list(mod_texts)
    dataset = tf.data.Dataset.from_tensor_slices((org_texts, mod_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [32]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (32, 20)
inputs["decoder_inputs"].shape: (32, 20)
targets.shape: (32, 20)


### 4.4 Building the model

Our sequence-to-sequence Transformer consists of a `TransformerEncoder`
and a `TransformerDecoder` chained together. To make the model aware of word order,
we also use a `PositionalEmbedding` layer.

The source sequence is passed to the `TransformerEncoder`,
which produces a new representation of it.
This new representation is then passed
to the `TransformerDecoder`, together with the target sequence so far (target words 0 to N).
The `TransformerDecoder` seeks to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking
(see method `get_causal_attention_mask()` on the `TransformerDecoder`).
The `TransformerDecoder` sees the entire sequences at once, and thus we must make
sure that it only uses information from target tokens 0 to N when predicting token N+1
(otherwise, it could use information from the future, which would
result in a model that cannot be used at inference time).  We have also referred to this as Masked Self-Attention.

In [33]:

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)



In [34]:
#Lets define our model
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

### 4.5 Training the model

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics, rather than accuracy.

Here we only train for 10 epochs, but to get better results we would need to feed it more data or find a way to train the encoder on Elizabethan English and to train the decoder on modern English.

In [35]:
epochs = 10  #

transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [36]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [37]:
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
418/418 [==============================] - 21s 42ms/step - loss: 3.0028 - accuracy: 0.2175 - val_loss: 2.6432 - val_accuracy: 0.2872
Epoch 2/10
418/418 [==============================] - 18s 42ms/step - loss: 2.5282 - accuracy: 0.3114 - val_loss: 2.4546 - val_accuracy: 0.3419
Epoch 3/10
418/418 [==============================] - 17s 41ms/step - loss: 2.2524 - accuracy: 0.3684 - val_loss: 2.3776 - val_accuracy: 0.3697
Epoch 4/10
418/418 [==============================] - 17s 41ms/step - loss: 2.0306 - accuracy: 0.4118 - val_loss: 2.3366 - val_accuracy: 0.3840
Epoch 5/10
418/418 [==============================] - 17s 41ms/step - loss: 1.8532 - accuracy: 0.4462 - val_loss: 2.3514 - val_accuracy: 0.3984
Epoch 6/10
418/418 [==============================] - 17s 41ms/step - loss: 1.6950 - accuracy: 0.4751 - val_loss: 2.3699 - val_accuracy: 0.4017
Epoch 7/10
418/418 [==============================] - 17s 42ms/step - loss: 1.5583 - accuracy: 0.5008 - val_loss: 2.3890 - val_accuracy:

### 4.6 Decoding some test sentences

Finally, let's demonstrate how to translate brand new sentences.
We simply feed into the model the vectorized Elizabethean English sentence
as well as the target token `"[start]"`, then we repeatedly generated the next token, until
we hit the token `"[end]"`.

In [38]:
mod_vocab = mod_vectorization.get_vocabulary()
mod_index_lookup = dict(zip(range(len(mod_vocab)), mod_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = org_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = mod_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = mod_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_org_texts = [pair[0] for pair in test_pairs]
for _ in range(10):
    input_sentence = random.choice(test_org_texts)
    translated = decode_sequence(input_sentence)
    print(f'in: {input_sentence}  ---->  out: {translated}')

in: O brave new world That has such people in't!  ---->  out: [start] o brave thing deed that has such a new lost it [end]
in: Who, in this kind of merry fooling, am nothing to you; so you may continue and laugh at nothing still.  ---->  out: [start] who is not safe for you yourself sir andrew it you can say it can say and you can delight
in: But why Stands Macbeth thus amazedly?  ---->  out: [start] but why is he at least that’s this storm [end]
in: Therefore to horse; And let us not be dainty of leave-taking, But shift away.  ---->  out: [start] so i think so don’t let us be silent of gentlemen [end]
in: Grant their lawful suit.  ---->  out: [start] i’ll their swaddling clothes [end]
in: I cannot go no further.  ---->  out: [start] i can’t go any further [end]
in: Where should Othello go?  ---->  out: [start] where should othello go [end]
in: The phrase would be more German to the matter if we could carry a cannon by our sides.  ---->  out: [start] the thing would be more for you as 

**QUESTION 2**: What things could we do to improve the output?
* add more sentence pairs
* ensure a good distribution over all the sentence lengths
* ???

[Return to Top](#returnToTop)  
<a id = 'translationMetrics'></a>

## 5. Machine Translation Metrics

In [39]:
import datasets
from datasets import list_datasets, list_metrics, load_dataset, load_metric
from pprint import pprint

In [40]:
metrics = list_metrics()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [41]:
print(f"🤩 Currently {len(metrics)} metrics are available on the hub:")
pprint(metrics, compact=True)

🤩 Currently 76 metrics are available on the hub:
['accuracy', 'bertscore', 'bleu', 'bleurt', 'brier_score', 'cer', 'chrf',
 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1',
 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis',
 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'pearsonr',
 'perplexity', 'poseval', 'precision', 'recall', 'rl_reliability', 'roc_auc',
 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2',
 'super_glue', 'ter', 'trec_eval', 'wer', 'wiki_split', 'xnli', 'xtreme_s',
 'GMFTBY/dailydialog_evaluate', 'GMFTBY/dailydialogevaluate',
 'Vertaix/vendiscore', 'Vlasta/pr_auc', 'abdusahmbzuai/aradiawer',
 'angelina-wang/directional_bias_amplification', 'cakiki/ndcg',
 'codeparrot/apps_metric', 'cpllab/syntaxgym', 'daiyizheng/valid',
 'erntkn/dice_coefficient', 'gorkaartola/metric_for_tp_fp_samples',
 'hack/test_metric', 'idsedykh/codebleu', 'idsedykh/codebleu2',
 'idsedykh/megaglue', 'i

### 5.1 BLEU example

The BLEU metric has been around for awhile. Let's run an example of the scoring using the function provided by the datasets library from HuggingFace.

In [42]:
#let's manually create some candidates and references
bleu_candidates = [
                   ["the", "earth", "trembled", "in", "Japan", "again", "on", "Monday", "the", "4th", "of", "September"], 
                   ["earthquakes", "struck", "Japan", "again", "on", "Monday", "the", "4th", "of", "September"]

]
bleu_references = [
                   [["earthquakes", "hit", "Japan", "again", "on", "Monday", "September", "4"]],
                   [["On", "September", "4th", "a", "Monday", "Japan", "had", "another", "earthquake"]]
]

In [43]:
from datasets import load_metric
#
bleu_metric = datasets.load_metric("bleu")


bleu_score = bleu_metric.compute(predictions=bleu_candidates, references=bleu_references)

print(bleu_score)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


{'bleu': 0.14367696612929734, 'precisions': [0.4090909090909091, 0.15, 0.1111111111111111, 0.0625], 'brevity_penalty': 1.0, 'length_ratio': 1.2941176470588236, 'translation_length': 22, 'reference_length': 17}


### 5.2 BLEURT

Now let's try the BLEURT metric.  It should score better than BLEU.  Again, think in terms of faithfulness and fluency.  How well does the candidate reflect the information in the reference? How well does the candidate read?

In [44]:
!pip install git+https://github.com/google-research/bleurt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-tb5lfmw9
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-tb5lfmw9
     |████████████████████████████████| 352 kB 35.1 MB/s 
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456761 sha256=5564001667e317d393451b3dd92f0ab92a2b508469f7c4b41839e240b3b4790f
  Stored in directory: /tmp/pip-ephem-wheel-cache-11wnqz5y/wheels/e2/2d/ea/b7a8b2424d2908d2a79d73ce8217d5ac4bd97ed3f47160a7f5
Successfully built BLEURT


In [47]:
#let's manually create some candidates and references
#candidates = ["the earth trembled in Japan again on Monday, the 4th of September", 
#              "earthquakes struck Japan again on Monday, the 4th of September",
#              "On September 4th, a Monday, Japan had another earthquake"]
candidates = ["the earth trembled in Japan again on Monday, the 4th of September"]
candidates2 = ["earthquakes shook Japan on Monday, the 4th of September"]
references = [["earthquakes hit Japan again on Monday, September 4"]]

In [49]:
#bleurt_metric = datasets.load_metric('bleurt') #this loads the smaller default model
bleurt_metric = datasets.load_metric('bleurt', 'bleurt-large-512')
bleurt_score = bleurt_metric.compute(predictions=candidates, references=references)

print(bleurt_score)

{'scores': [-0.022389188408851624]}


Notice that the candidate and reference don't get a very good score. Do you think they should get a higher score?

Try running it again except this time use candidates2 and see what kind of score it gets.  Is that what you expected?

[Return to Top](#returnToTop)  
<a id = 'subwordModels'></a>

## 6. Subword Models

Different pretrained models use different subword models.  Each subword model identifies a different set of "tokens" based on an efficient representation of words and parts of words in the pre-training corpus.  The model has an embedding for each one of the subwords in its vocabulary.

We do not typically interact directly with the subword models but rather do so indirectly through the Tokenizer object.

Let's try the BERT base cased tokenizer.  It uses a wordpiece subword model. 

In [50]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

print(f'The vocabulary size is {tokenizer.vocab_size}')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

The vocabulary size is 28996


In [51]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'Transformers',
 '?',
 'We',
 'sure',
 'do',
 '.']

This is the same tokenizer code but instead it is loaded with the multilingual model version. Note that it contains many more tokens than BERT base because it has to be able to deal with multiple kinds of symbols.

In [52]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
print(f'The vocabulary size is {tokenizer.vocab_size}')

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

The vocabulary size is 105879


In [ ]:
tokenizer.tokenize("你不喜欢🤗变形金刚吗？ 我们肯定会。")

['你',
 '不',
 '喜',
 '欢',
 '[UNK]',
 '变',
 '形',
 '金',
 '刚',
 '吗',
 '？',
 '我',
 '们',
 '肯',
 '定',
 '会',
 '。']

Let's put that first English sentence through the multilingual tokenizer.  It produces the same subwords for English even though it can also handle other languages as shown by it's much lager vocabulary size.

In [53]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'transformers',
 '?',
 'we',
 'sure',
 'do',
 '.']

T5 uses the sentencepiece subword model.  Here we'll use the tokenizer for the multilingual version of T5 called mt5.  Notice the vocabulary size.

In [54]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
print(f'The vocabulary size is {tokenizer.vocab_size}')

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

The vocabulary size is 250100


In [55]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁',
 'Transformers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

The sentencepiece subword model includes a marker to indicate if a subword is at the begining of a word and thus, in English, is preceeded by a space.  This means that with sentence piece it is possible to accurtely reconstruct the sentence because we explicitly identify the word boundaries.


Finally, let's look at **GPT2** which uses the BytePair Encoding subword model.  Its output will be completely different.

In [56]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(f'The vocabulary size is {tokenizer.vocab_size}')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

The vocabulary size is 50257


In [57]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'t",
 'Ġyou',
 'Ġlove',
 'ĠðŁ',
 '¤',
 'Ĺ',
 'ĠTransformers',
 '?',
 'ĠWe',
 'Ġsure',
 'Ġdo',
 '.']

[Return to Top](#returnToTop)  
<a id = 'answers'></a>

## ANSWERS

1.  The T5 model doesn't have the token type ids that BERT uses to identify different segments.

2.  The first two suggestions -- more sentence pairs and better balance on length are a good start.  More and better data typically lead to improved performance.  We might also look into separtely "pre-training" our encoder and decoder with their own language models.  We could then use those as pre-trained models as a foundation on which we train our connected encoder and decoder.  We could also look in to using back translation to augment our existing sentence pairs. 
